In [ ]:
import numpy as np
import statsmodels.api as sm
import scipy as sp
import scipy.special

In [ ]:
# sigmoid function
def sigmoid(x):
    return (1/(1 + np.exp(-x)))

# weight dot independent variables 
def pro(w, x):
    return sigmoid(np.inner(w, x))

# first coefficient in beta prior
def alpha(p, phi):
    return (((1 / phi) - 1) * p)

# second coefficient in beta prior
def beta(p, phi):
    return (((1 / phi) - 1) * (1 - p))

# average cost of beta-binomial regression excluding constant 
def cost(f, exog, endog):
    cos = 0
    for n in range(len(exog)):
        p = pro(f[:-1], exog[n])
        a = alpha(p, f[-1])
        b = beta(p, f[-1])
        cos += sp.special.betaln(endog[n][0] + a, endog[n][1] + b) - sp.special.betaln(a, b)
    return - cos / len(exog)

# predict the output given the parametres
def predict(f, exog, endog):
    p_pred = sigmoid(np.matmul(exog, f[:-1]))
    return np.concatenate(([p_pred * (endog[:, 0] + endog[:, 1])], [(1 - p_pred) * (endog[:, 0] + endog[:, 1])]), axis = 0).T 

# log-likelihood of the data
def LL(f, exog, endog):
    return - cost(f, exog, endog) * len(exog) + sum(sp.special.gammaln(endog[:,0] + endog[:,1] + 1) - sp.special.gammaln(endog[:,0] + 1) - sp.special.gammaln(endog[:,1] + 1))

# find the best initial guess for phi
def initial(params, exog, endog):
    guess = np.arange(0.99999, 10, 0.1)
    cos = [0]
    while np.argmin(cos) == 0:
        guess = guess / 10
        cos = []
        for m in range(len(guess)): 
            f = np.concatenate((params, [guess[m]]))
            cos.append(cost(f, exog, endog))
    f = np.concatenate((params, [guess[np.argmin(cos)]]))
    # print("initial guess = ", f)
    # print("cost of initial guess =", cos[np.argmin(cos)])
    return f

# beta-binomial regression
def fit(self):
    glm_binom = sm.GLM(self.endog, self.exog, family = sm.families.Binomial())
    res = glm_binom.fit()
        
    bnds = []
    for m in range(len(res.params)):
        bnds.append((- np.inf, np.inf))
    bnds.append((0,1))
    bnds_t = tuple(bnds)
        
    f = initial(res.params, self.exog, self.endog)
    ress = sp.optimize.minimize(cost, f, args = (self.exog, self.endog), method = self.method, bounds = bnds_t)
    return ress.x

In [ ]:
# can be any method scipy.optimize.minimize supports
class betabin:
    def __init__(self, exog, endog, fit_intercept = True, method = "SLSQP"):
        self.n_features_in_ = len(exog[0])
        self.intercept_ = fit_intercept
        if self.intercept_:
            exog = sm.add_constant(exog, prepend = False)
        self.exog = exog
        self.endog = endog
        self.method = method
        self.params = fit(self)
        self.predict = predict(self.params, self.exog, self.endog)
        self.LL = LL(self.params, self.exog, self.endog)